In [1]:
# Use the Training data and create the model and data drift detector object and save it.

# Then create a schedule which runs after every specific period that checks the data from the logging table for drift and raises a trigger

# If trigger happens then all the data till 1 week before should be queried from the logging table again
# and model should be retrained. Then the new model & old model should test on the last 1 week data for selecting the best.

# Once the final model is selected, the old model pickle file should be pushed to archive and the new model should take its position along with all the other features and mappings


In [2]:
%%capture
!pip install alibi

In [3]:
%%capture
!pip install alibi-detect

In [4]:
%%capture

!pip install snowflake-connector-python

In [5]:
%%capture

!pip install snowflake-sqlalchemy

In [6]:
%%capture

!pip install xgboost

In [7]:
! pip install pandas=="2.1.4"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autovizwidget 0.21.0 requires pandas<2.0.0,>=0.20.1, but you have pandas 2.1.4 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.


In [8]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle 

import sklearn
from sklearn import metrics

import alibi 
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('America/Indianapolis')


import snowflake_creds

import warnings
warnings.filterwarnings('ignore')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
engine = create_engine(URL(
         account = "kbxyguy-yh77168",
        #account="by44730.us-east-2.aws",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
    ))

## Creating the Model and Data drift detector object from Training set:

#### Data Drift detector:

In [10]:
query = """

SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.PUBLIC.HEALTH_DATA

"""

In [11]:
# Loading the train data
with engine.connect() as conn:
    df_train = pd.DataFrame(pd.read_sql(query,conn))
    df_train.columns = [col.upper() for col in df_train.columns.tolist()]

In [12]:
print(df_train.shape)
df_train.head(3)

(308435, 19)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE
0,1,19,a,7,Y,4,gynecology,Q,C,3,48585,8,Emergency,Moderate,2,61-70,3827,2022-12-01,2022-12-31
1,2,26,b,2,Y,5,gynecology,R,D,2,48585,8,Urgent,Moderate,4,61-70,4677,2022-12-01,2023-01-10
2,3,19,a,7,Y,2,gynecology,S,C,2,110115,8,Emergency,Moderate,4,71-80,3924,2022-12-01,2023-01-30


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308435 entries, 0 to 308434
Data columns (total 19 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   CASE_ID                            308435 non-null  int64 
 1   HOSPITAL_CODE                      308435 non-null  int64 
 2   HOSPITAL_TYPE_CODE                 308435 non-null  object
 3   CITY_CODE_HOSPITAL                 308435 non-null  int64 
 4   HOSPITAL_REGION_CODE               308435 non-null  object
 5   AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL  308435 non-null  int64 
 6   DEPARTMENT                         308435 non-null  object
 7   WARD_TYPE                          308435 non-null  object
 8   WARD_FACILITY_CODE                 308435 non-null  object
 9   BED_GRADE                          308435 non-null  int64 
 10  PATIENTID                          308435 non-null  int64 
 11  CITY_CODE_PATIENT                  308435 non-null  

In [14]:
# Getting the numerical and categorical columns for creating the datadrift object
num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
cat_columns = [col for col in df_train.columns.tolist() if col not in num_columns+id_columns]

In [15]:
num_columns

['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL',
 'VISITORS_WITH_PATIENT',
 'ADMISSION_DEPOSIT']

In [16]:
cat_columns

['HOSPITAL_CODE',
 'HOSPITAL_TYPE_CODE',
 'CITY_CODE_HOSPITAL',
 'HOSPITAL_REGION_CODE',
 'DEPARTMENT',
 'WARD_TYPE',
 'WARD_FACILITY_CODE',
 'BED_GRADE',
 'CITY_CODE_PATIENT',
 'TYPE_OF_ADMISSION',
 'SEVERITY_OF_ILLNESS',
 'AGE']

In [17]:
X_train = df_train[num_columns + cat_columns]
print(X_train.shape)
X_train.head()

(308435, 15)


,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,AGE
0,4,2,3827,19,a,7,Y,gynecology,Q,C,3,8,Emergency,Moderate,61-70
1,5,4,4677,26,b,2,Y,gynecology,R,D,2,8,Urgent,Moderate,61-70
2,2,4,3924,19,a,7,Y,gynecology,S,C,2,8,Emergency,Moderate,71-80
3,2,2,5124,18,d,13,Y,radiotherapy,R,B,2,3,Emergency,Moderate,31-40
4,5,4,4527,23,a,6,X,gynecology,Q,F,4,2,Trauma,Moderate,21-30


In [18]:
cat_indices = np.arange(3,15)
cat_indices

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [19]:
# category dict for the driftdetector to identify unique categories
categories_per_feature = {f: None for f in cat_indices}
categories_per_feature

{3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None}

In [20]:
# Initialize the detector
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)

In [21]:
cd

TabularDrift

In [22]:
cd.get_config()

{'name': 'TabularDrift',
 'meta': {'version': '0.12.0'},
 'x_ref': array([[4, 2, 3827, ..., 'Emergency', 'Moderate', '61-70'],
        [5, 4, 4677, ..., 'Urgent', 'Moderate', '61-70'],
        [2, 4, 3924, ..., 'Emergency', 'Moderate', '71-80'],
        ...,
        [3, 15, 4241, ..., 'Emergency', 'Moderate', '41-50'],
        [4, 4, 3036, ..., 'Emergency', 'Extreme', '41-50'],
        [4, 4, 4326, ..., 'Trauma', 'Extreme', '31-40']], dtype=object),
 'p_val': 0.05,
 'categories_per_feature': {3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None,
  12: None,
  13: None,
  14: None},
 'x_ref_preprocessed': False,
 'preprocess_at_init': True,
 'update_x_ref': None,
 'preprocess_fn': None,
 'correction': 'bonferroni',
 'alternative': 'two-sided',
 'n_features': None,
 'input_shape': None,
 'data_type': None}

In [23]:
# # Using pickle to save and load it the trained detector
with open('Trained_Drift_Detector.pkl','wb') as F:
     pickle.dump(cd,F)

with open('Trained_Drift_Detector.pkl','rb') as F:
    trained_drift_model = pickle.load(F)    

In [24]:
trained_drift_model.get_config()

{'name': 'TabularDrift',
 'meta': {'version': '0.12.0'},
 'x_ref': array([[4, 2, 3827, ..., 'Emergency', 'Moderate', '61-70'],
        [5, 4, 4677, ..., 'Urgent', 'Moderate', '61-70'],
        [2, 4, 3924, ..., 'Emergency', 'Moderate', '71-80'],
        ...,
        [3, 15, 4241, ..., 'Emergency', 'Moderate', '41-50'],
        [4, 4, 3036, ..., 'Emergency', 'Extreme', '41-50'],
        [4, 4, 4326, ..., 'Trauma', 'Extreme', '31-40']], dtype=object),
 'p_val': 0.05,
 'categories_per_feature': {3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None,
  12: None,
  13: None,
  14: None},
 'x_ref_preprocessed': False,
 'preprocess_at_init': True,
 'update_x_ref': None,
 'preprocess_fn': None,
 'correction': 'bonferroni',
 'alternative': 'two-sided',
 'n_features': None,
 'input_shape': None,
 'data_type': None}

In [25]:
X_train.values

array([[4, 2, 3827, ..., 'Emergency', 'Moderate', '61-70'],
       [5, 4, 4677, ..., 'Urgent', 'Moderate', '61-70'],
       [2, 4, 3924, ..., 'Emergency', 'Moderate', '71-80'],
       ...,
       [3, 15, 4241, ..., 'Emergency', 'Moderate', '41-50'],
       [4, 4, 3036, ..., 'Emergency', 'Extreme', '41-50'],
       [4, 4, 4326, ..., 'Trauma', 'Extreme', '31-40']], dtype=object)

In [26]:
fpreds = trained_drift_model.predict(X_train.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[fpreds['data']['is_drift']]))

Drift? No!


In [27]:
fpreds['data']['is_drift']

0

In [28]:
# individual feature-wise drift, this is also possible:
feature_preds = trained_drift_model.predict(X_train.values, drift_type='feature')
feature_preds['data']['distance']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [29]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_train.columns.tolist()[f]
    # print(f, fname)
    is_drift = feature_preds['data']['is_drift'][f]
    stat_val, p_val = feature_preds['data']['distance'][f], feature_preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL -- Drift? No! -- K-S 0.000 -- p-value 1.000
VISITORS_WITH_PATIENT -- Drift? No! -- K-S 0.000 -- p-value 1.000
ADMISSION_DEPOSIT -- Drift? No! -- K-S 0.000 -- p-value 1.000
HOSPITAL_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_TYPE_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_HOSPITAL -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_REGION_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
DEPARTMENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_TYPE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_FACILITY_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
BED_GRADE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_PATIENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
TYPE_OF_ADMISSION -- Drift? No! -- Chi2 0.000 -- p-value 1.000
SEVERITY_OF_ILLNESS -- Drift? No! -- Chi2 0.000 -- p-value 1.000
AGE -- Drift? No! -- Chi2 0.000 -- p-value 1.000


In [30]:
temp = pd.DataFrame()
temp['Time Period'] = [str('2023-01-01') + ' to ' + str('2023-01-07')]*len(X_train.columns.tolist())
temp['Features'] = X_train.columns.tolist()
temp['Is Drift'] = fpreds['data']['is_drift']
temp['Stat Test'] = temp['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
temp['Stats Value'] = fpreds['data']['distance']
temp['P-value'] = fpreds['data']['p_val']
print(temp.shape)
temp

(15, 6)


,Time Period,Features,Is Drift,Stat Test,Stats Value,P-value
0,2023-01-01 to 2023-01-07,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0,K-S,0.0,1.0
1,2023-01-01 to 2023-01-07,VISITORS_WITH_PATIENT,0,K-S,0.0,1.0
2,2023-01-01 to 2023-01-07,ADMISSION_DEPOSIT,0,K-S,0.0,1.0
3,2023-01-01 to 2023-01-07,HOSPITAL_CODE,0,Chi2,0.0,1.0
4,2023-01-01 to 2023-01-07,HOSPITAL_TYPE_CODE,0,Chi2,0.0,1.0
5,2023-01-01 to 2023-01-07,CITY_CODE_HOSPITAL,0,Chi2,0.0,1.0
6,2023-01-01 to 2023-01-07,HOSPITAL_REGION_CODE,0,Chi2,0.0,1.0
7,2023-01-01 to 2023-01-07,DEPARTMENT,0,Chi2,0.0,1.0
8,2023-01-01 to 2023-01-07,WARD_TYPE,0,Chi2,0.0,1.0
9,2023-01-01 to 2023-01-07,WARD_FACILITY_CODE,0,Chi2,0.0,1.0


##### Creating a noise in the train data to check if drift detector is working

In [31]:
temp = X_train.copy()
temp.loc[:50,'HOSPITAL_CODE'] = 100
temp.head()

,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,AGE
0,4,2,3827,100,a,7,Y,gynecology,Q,C,3,8,Emergency,Moderate,61-70
1,5,4,4677,100,b,2,Y,gynecology,R,D,2,8,Urgent,Moderate,61-70
2,2,4,3924,100,a,7,Y,gynecology,S,C,2,8,Emergency,Moderate,71-80
3,2,2,5124,100,d,13,Y,radiotherapy,R,B,2,3,Emergency,Moderate,31-40
4,5,4,4527,100,a,6,X,gynecology,Q,F,4,2,Trauma,Moderate,21-30


In [32]:
# If  interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(temp.values, drift_type='feature')
fpreds

{'data': {'is_drift': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([ 0.      ,  0.      ,  0.      , 51.007603,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ], dtype=float32),
  'p_val': array([1.        , 1.        , 1.        , 0.01778425, 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ],
        dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.12.0',
  'detector_type': 'drift'}}

In [33]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = temp.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL -- Drift? No! -- K-S 0.000 -- p-value 1.000
VISITORS_WITH_PATIENT -- Drift? No! -- K-S 0.000 -- p-value 1.000
ADMISSION_DEPOSIT -- Drift? No! -- K-S 0.000 -- p-value 1.000
HOSPITAL_CODE -- Drift? Yes! -- Chi2 51.008 -- p-value 0.018
HOSPITAL_TYPE_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_HOSPITAL -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_REGION_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
DEPARTMENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_TYPE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_FACILITY_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
BED_GRADE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_PATIENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
TYPE_OF_ADMISSION -- Drift? No! -- Chi2 0.000 -- p-value 1.000
SEVERITY_OF_ILLNESS -- Drift? No! -- Chi2 0.000 -- p-value 1.000
AGE -- Drift? No! -- Chi2 0.000 -- p-value 1.000


#### Data Drift Scoring:

In [34]:
def data_monitoring_batch_query(a):
    query = f"""

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE("AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_x",0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE("VISITORS_WITH_PATIENT_x",0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE("ADMISSION_DEPOSIT_x",0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

            FROM HEALTHDB.PUBLIC.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED
            WHERE ADMISSION_DATE >= CURRENT_DATE-562+{a*7} AND ADMISSION_DATE < CURRENT_DATE-562+{(a+1)*7}        

        """
    return query

In [35]:
print(data_monitoring_batch_query(0))
#TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED



        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE("AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_x",0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE("VISITORS_WITH_PATIENT_x",0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'No

In [37]:
def data_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(data_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
    # Getting the numerical and categorical columns for creating the datadrift object
    num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
    id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
    cat_columns = [col for col in batch_df.columns.tolist() if col not in num_columns+id_columns]
    
    # Getting the final prepared data
    batch_final = batch_df[num_columns + cat_columns]
    
    # Loading the Trained data drift detector
    with open('Trained_Drift_Detector.pkl','rb') as F:
        trained_drift_model = pickle.load(F)    
    
    # Checking for drift
    # If  individual feature-wise drift"
    fpreds = trained_drift_model.predict(batch_final.values, drift_type='feature')
    
    log_df = pd.DataFrame()
    log_df['Time Period'] = ([str(batch_df['ADMISSION_DATE'].min()) + ' to ' + 
                              str(batch_df['ADMISSION_DATE'].max())]
                              * len(batch_final.columns.tolist())
                            )
    log_df['Total Records'] = batch_df.shape[0]
    log_df['Features'] = batch_final.columns.tolist()
    log_df['Is Drift'] = fpreds['data']['is_drift']
    log_df['Stat Test'] = log_df['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
    log_df['Stats Value'] = np.round(fpreds['data']['distance'])
    log_df['P-value'] = np.round(fpreds['data']['p_val'])
    
    return log_df

In [38]:
data_monitoring(0)

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-01 to 2022-12-07,16842,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-01 to 2022-12-07,16842,VISITORS_WITH_PATIENT,0,K-S,0.0,0.0
2,2022-12-01 to 2022-12-07,16842,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-01 to 2022-12-07,16842,HOSPITAL_CODE,1,Chi2,100.0,0.0
4,2022-12-01 to 2022-12-07,16842,HOSPITAL_TYPE_CODE,1,Chi2,27.0,0.0
5,2022-12-01 to 2022-12-07,16842,CITY_CODE_HOSPITAL,1,Chi2,29.0,0.0
6,2022-12-01 to 2022-12-07,16842,HOSPITAL_REGION_CODE,0,Chi2,1.0,1.0
7,2022-12-01 to 2022-12-07,16842,DEPARTMENT,1,Chi2,12.0,0.0
8,2022-12-01 to 2022-12-07,16842,WARD_TYPE,1,Chi2,25.0,0.0
9,2022-12-01 to 2022-12-07,16842,WARD_FACILITY_CODE,0,Chi2,3.0,1.0


In [39]:
t = data_monitoring(0)
t

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-01 to 2022-12-07,16842,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-01 to 2022-12-07,16842,VISITORS_WITH_PATIENT,0,K-S,0.0,0.0
2,2022-12-01 to 2022-12-07,16842,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-01 to 2022-12-07,16842,HOSPITAL_CODE,1,Chi2,100.0,0.0
4,2022-12-01 to 2022-12-07,16842,HOSPITAL_TYPE_CODE,1,Chi2,27.0,0.0
5,2022-12-01 to 2022-12-07,16842,CITY_CODE_HOSPITAL,1,Chi2,29.0,0.0
6,2022-12-01 to 2022-12-07,16842,HOSPITAL_REGION_CODE,0,Chi2,1.0,1.0
7,2022-12-01 to 2022-12-07,16842,DEPARTMENT,1,Chi2,12.0,0.0
8,2022-12-01 to 2022-12-07,16842,WARD_TYPE,1,Chi2,25.0,0.0
9,2022-12-01 to 2022-12-07,16842,WARD_FACILITY_CODE,0,Chi2,3.0,1.0


In [40]:
print(t.shape[0])
print(t['Is Drift'].sum())

15
12


### Model Drift detector:

In [41]:
# Get the data from the logging table in batches (7 days)
# Using the predicted and actual LOS calculate the performance metrics dict
# Then use the ref_metric_dict (from training) to compare with the current_metric_dict for model drift

In [42]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
         account = "kbxyguy-yh77168",
        #account="by44730.us-east-2.aws",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
    ))

In [43]:
# Function to check model drift
def check_model_drift(ref_metric_dict,cur_metric_dict,type='classification',tol=0.1):
    if type == 'classification':
        precision_change = abs((cur_metric_dict['Precision']-ref_metric_dict['Precision'])/ref_metric_dict['Precision'])
        recall_change = abs((cur_metric_dict['Recall']-ref_metric_dict['Recall'])/ref_metric_dict['Recall'])
        roc_auc_change = abs((cur_metric_dict['Roc-Auc']-ref_metric_dict['Roc-Auc'])/ref_metric_dict['Roc-Auc'])

        counter = 0
        for i in [precision_change,recall_change,roc_auc_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            print("Change in Precision: "+ str(np.round(100*precision_change,2))+"%")
            print("Change in Recall: "+ str(np.round(100*recall_change,2))+"%")
            print("Change in Roc-Auc: "+ str(np.round(100*roc_auc_change,2))+"%")
            return 1
        else:
            print("There is no model drift.")
            return 0

    elif type == 'regression':
        rmse_change = abs((cur_metric_dict['RMSE']-ref_metric_dict['RMSE'])/ref_metric_dict['RMSE'])
        mae_change = abs((cur_metric_dict['MAE']-ref_metric_dict['MAE'])/ref_metric_dict['MAE'])
        
        counter = 0
        for i in [rmse_change,mae_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            RMSE_CHANGE = np.round(100*rmse_change,2)
            MAE_CHANGE = np.round(100*mae_change,2)
            print("Change in RMSE: "+ str(np.round(100*rmse_change,2))+"%")
            print("Change in MAE: "+ str(np.round(100*mae_change,2))+"%")
            return 1, RMSE_CHANGE, MAE_CHANGE
        else:
            print("There is no model drift.")
            RMSE_CHANGE = 'NONE'
            MAE_CHANGE = 'NONE'
            return 0, RMSE_CHANGE, MAE_CHANGE
        
        

In [46]:
def model_monitoring_batch_query(a):
    query_sim = f"""

        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED
        WHERE ADMISSION_DATE >= CURRENT_DATE-562+{a*7} AND ADMISSION_DATE < CURRENT_DATE-562+{(a+1)*7}
        
    """
    return query_sim

In [47]:
print(model_monitoring_batch_query(0))



        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED
        WHERE ADMISSION_DATE >= CURRENT_DATE-562+0 AND ADMISSION_DATE < CURRENT_DATE-562+7
        
    


In [48]:
# Loading the train data
with engine.connect() as conn:
    batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(0),conn))
    batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]

In [50]:
print(batch_df.shape)
batch_df.head()

(16842, 47)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT_X,AGE,ADMISSION_DEPOSIT_X,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS_X,BED_GRADE_2,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_Y,CITY_CODE_HOSPITAL_2,ADMISSION_MONTH_NOV,AGE_31-40,WARD_TYPE_S,ADMISSION_DEPOSIT_Y,ILLNESS_BEDGRADE_EXTREME-1,WARD_TYPE_P,AGE_71-80,VISITORS_WITH_PATIENT_Y,AGE_51-60,SEVERITY_OF_ILLNESS_MINOR,CITY_CODE_PATIENT_8,CITY_CODE_HOSPITAL_7,TYPE_OF_ADMISSION_TRAUMA,ADMISSION_MONTH_OCT,AGE_41-50,WARD_TYPE_Q,TYPE_OF_ADMISSION_EMERGENCY,LOS_Y,PREDICTED_LOS
0,1,19,a,7,Y,4,gynecology,Q,C,3,48585,8,Emergency,Moderate,2,61-70,3827,2022-12-01,2022-12-31,Dec,Thu,Emergency-Moderate,Moderate-3,gynecology-Moderate,30,False,4,False,0,False,False,3827,False,False,False,2,False,False,True,True,False,0,False,True,True,30,23.0
1,2,26,b,2,Y,5,gynecology,R,D,2,48585,8,Urgent,Moderate,4,61-70,4677,2022-12-01,2023-01-10,Dec,Thu,Urgent-Moderate,Moderate-2,gynecology-Moderate,40,True,5,True,0,False,False,4677,False,False,False,4,False,False,True,False,False,0,False,False,False,40,58.0
2,3,19,a,7,Y,2,gynecology,S,C,2,110115,8,Emergency,Moderate,4,71-80,3924,2022-12-01,2023-01-30,Dec,Thu,Emergency-Moderate,Moderate-2,gynecology-Moderate,60,True,2,False,0,False,True,3924,False,False,True,4,False,False,True,True,False,0,False,False,True,60,45.0
3,4,18,d,13,Y,2,radiotherapy,R,B,2,127970,3,Emergency,Moderate,2,31-40,5124,2022-12-01,2022-12-11,Dec,Thu,Emergency-Moderate,Moderate-2,radiotherapy-Moderate,10,True,2,False,0,True,False,5124,False,False,False,2,False,False,False,False,False,0,False,False,True,10,27.0
4,5,23,a,6,X,5,gynecology,Q,F,4,130244,2,Trauma,Moderate,4,21-30,4527,2022-12-01,2022-12-21,Dec,Thu,Trauma-Moderate,Moderate-4,gynecology-Moderate,20,False,5,False,0,False,False,4527,False,False,False,4,False,False,False,False,True,0,False,True,False,20,29.0


In [51]:
# Creating the current performance dict (from scoring)

actual = batch_df['LOS_X']
predicted = batch_df['PREDICTED_LOS']

rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
print("RMSE: ", rmse)
print("MAE: ", mae)

scoring_ref_metrics = {}
scoring_ref_metrics['RMSE'] = rmse
scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
print(scoring_ref_metrics)

RMSE:  14.507592940907905
MAE:  3.2690027974705926
{'RMSE': 14.507592940907905, 'MAE': 3.2690027974705926}


In [57]:
# I tried to loading the reference performance dict (from training) but then realized I havent created one, so just creating a 
#dictionary manually

file_path = 'MODEL_XGB_PERFM_METRICS.pkl'
model_ref_metric = {'RMSE': 13.868612241581472,
'MAE':  3.195693747437704}


# Writing data to the file
with open(file_path, 'wb') as file:
    pickle.dump(model_ref_metric, file)

print(f"Model metrics saved to {file_path}")

with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
    model_ref_metric = pickle.load(F)

#model_ref_metric

# Creating the ref script manually


Model metrics saved to MODEL_XGB_PERFM_METRICS.pkl


In [58]:
check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)

There is no model drift.


(0, 'NONE', 'NONE')

In [59]:
def model_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
#     print(batch_df.shape)
    
    # Creating the current performance dict (from scoring)
    actual = batch_df['LOS_X']
    predicted = batch_df['PREDICTED_LOS']

    rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
    mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
#     print("RMSE: ", rmse)
#     print("MAE: ", mae)

    scoring_ref_metrics = {}
    scoring_ref_metrics['RMSE'] = rmse
    scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
#     print(scoring_ref_metrics)
    
    
    # Loading the reference performance dict (from training)
    with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
        model_ref_metric = pickle.load(F)
        
#     print(model_ref_metric)
    
    # Check for model drift
    model_drift, RMSE_CHANGE, MAE_CHANGE = check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)
    
    # Log values
    log = {}
    log['Time Period'] = str(batch_df['ADMISSION_DATE'].min()) + ' to ' + str(batch_df['ADMISSION_DATE'].max())
    log['Total Records'] = batch_df.shape[0]
    log['Scoring Metrics'] = scoring_ref_metrics
    log['Training Metrics'] = model_ref_metric
    log['Model Drift IND'] = model_drift
    log['RMSE Change'] = RMSE_CHANGE
    log['MAE Change'] = MAE_CHANGE
    
    return log
    

In [71]:
# It words till 4 weeks, since the mentioned range is from 1 dec 2022 to 31 dec 2022
model_monitoring(4)

There is no model drift.


{'Time Period': '2022-12-29 to 2022-12-30',
 'Total Records': 4882,
 'Scoring Metrics': {'RMSE': 14.704998913146547, 'MAE': 3.2550027673041573},
 'Training Metrics': {'RMSE': 13.868612241581472, 'MAE': 3.195693747437704},
 'Model Drift IND': 0,
 'RMSE Change': 'NONE',
 'MAE Change': 'NONE'}

In [ ]:
# Create a dictionary with has two keys --> {data_drift:'Yes', model_drift:'Yes'}
# Based on the logic above, it should call the retraining function below 
# Once above steps are completed, create the retraining script that takes in all the data from the training and logging table (except the last 1 or 2 week)

# Model Monitoring & Retraining Pipeline:

In [77]:
data_log_df = data_monitoring(1)
model_log_dict = model_monitoring(1)

There is no model drift.


In [78]:
data_log_df

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-08 to 2022-12-14,16578,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-08 to 2022-12-14,16578,VISITORS_WITH_PATIENT,0,K-S,0.0,0.0
2,2022-12-08 to 2022-12-14,16578,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-08 to 2022-12-14,16578,HOSPITAL_CODE,1,Chi2,61.0,0.0
4,2022-12-08 to 2022-12-14,16578,HOSPITAL_TYPE_CODE,0,Chi2,8.0,0.0
5,2022-12-08 to 2022-12-14,16578,CITY_CODE_HOSPITAL,1,Chi2,21.0,0.0
6,2022-12-08 to 2022-12-14,16578,HOSPITAL_REGION_CODE,0,Chi2,0.0,1.0
7,2022-12-08 to 2022-12-14,16578,DEPARTMENT,1,Chi2,19.0,0.0
8,2022-12-08 to 2022-12-14,16578,WARD_TYPE,1,Chi2,27.0,0.0
9,2022-12-08 to 2022-12-14,16578,WARD_FACILITY_CODE,0,Chi2,6.0,0.0


In [79]:
# Data drift condition
data_log_df['Is Drift'].sum() > 0

True

In [80]:
model_log_dict

{'Time Period': '2022-12-08 to 2022-12-14',
 'Total Records': 16578,
 'Scoring Metrics': {'RMSE': 14.668219162395518, 'MAE': 3.26889396469861},
 'Training Metrics': {'RMSE': 13.868612241581472, 'MAE': 3.195693747437704},
 'Model Drift IND': 0,
 'RMSE Change': 'NONE',
 'MAE Change': 'NONE'}

In [81]:
# Model drift condition
model_log_dict['Model Drift IND']

0

In [82]:
# Max date for retraining 
max_date = model_log_dict['Time Period'].split(' ')[2]
max_date

'2022-12-14'